In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Parameters from the problem
S = 30           # Spot price
sigma = 0.30     # Volatility
q = 0.01         # Dividend yield
r = 0.025        # Risk-free rate
T = 0.25         # Time to maturity in years (3 months)
tol = 1e-6       # Tolerance for Newton's method
target_delta = 0.5

# Delta as a function of strike K
def delta_call(K):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return np.exp(-q * T) * norm.cdf(d1)

# Derivative of delta with respect to strike K (for Newton's method)
def ddelta_dK(K):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    dN_dd1 = norm.pdf(d1)
    dd1_dK = -1 / (K * sigma * np.sqrt(T))
    return np.exp(-q * T) * dN_dd1 * dd1_dK

# Newton's method
def newton_method(f, df, x0, tol):
    iterations = []
    x = x0
    while True:
        fx = f(x)
        dfx = df(x)
        x_new = x - (fx - target_delta) / dfx
        iterations.append((x, fx, dfx, x_new))
        if abs(x_new - x) < tol:
            break
        x = x_new
    return pd.DataFrame(iterations, columns=['K', 'Delta(K)', "dDelta/dK", 'Next K'])

# Start with ATM strike
K0 = S
results_df = newton_method(delta_call, ddelta_dK, K0, tol)
results_df


,K,Delta(K),dDelta/dK,Next K
0,30.000000,0.538480,-0.087991,30.437315
1,30.437315,0.500153,-0.087161,30.439064
2,30.439064,0.500000,-0.087156,30.439065


In [2]:
results_df

,K,Delta(K),dDelta/dK,Next K
0,30.000000,0.538480,-0.087991,30.437315
1,30.437315,0.500153,-0.087161,30.439064
2,30.439064,0.500000,-0.087156,30.439065


In [7]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Parameters
S = 30           # Spot price
sigma = 0.30     # Volatility
q = 0.01         # Dividend yield
r = 0.025        # Risk-free rate
T = 0.25         # Time to maturity (in years)
tol = 1e-6       # Tolerance for Newton's method
target_delta = 0.5

# Delta as a function of strike K
def delta_call(K):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return np.exp(-q * T) * norm.cdf(d1)

# Manual derivative of Delta with respect to strike K
def ddelta_dK(K):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    pdf_d1 = (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * d1**2)
    dd1_dK = -1 / (K * sigma * np.sqrt(T))
    return np.exp(-q * T) * pdf_d1 * dd1_dK

# Newton's method with error computation
def newton_method(f, df, x0, tol):
    iterations = []
    x = x0
    while True:
        fx = f(x)
        dfx = df(x)
        x_new = x - (fx - target_delta) / dfx
        error = abs(x_new - x)
        iterations.append((x, fx, dfx, x_new, error))
        if error < tol:
            break
        x = x_new
    return pd.DataFrame(iterations, columns=['K', 'Delta(K)', 'dDelta/dK', 'Next K', 'Error'])

# Initial guess: ATM strike
K0 = S
results_df = newton_method(delta_call, ddelta_dK, K0, tol)

# Display LaTeX version
latex_output = results_df.to_latex(
    caption='Newton Method Iterations for Finding Strike starting from ATM',
    label='tab:newton_iterations',
    index=False
)


In [8]:
print(latex_output)

\begin{table}
\caption{Newton Method Iterations for Finding Strike starting from ATM}
\label{tab:newton_iterations}
\begin{tabular}{rrrrr}
\toprule
K & Delta(K) & dDelta/dK & Next K & Error \\
\midrule
30.000000 & 0.538480 & -0.087991 & 30.437315 & 0.437315 \\
30.437315 & 0.500153 & -0.087161 & 30.439064 & 0.001750 \\
30.439064 & 0.500000 & -0.087156 & 30.439065 & 0.000000 \\
\bottomrule
\end{tabular}
\end{table}

